# SELECCIÓ DEL MODEL:
Ara ja tenim les dades llestes per experimentar amb models. Tots els models probats han estat probats i executats en 70% train, 30% test i amb el cross-validation k-fold, amb k=5.

He agafat una llista de 5 models per fer una comparativa i veure quin d'aquests dona millors resultats:
- RandomForest
- Màquina de vectors de suport Lineal
- Naive Bayes Multinomial
- Naive Bayes Gaussià
- Regressió Logística

In [37]:
from sklearn.svm import SVC, NuSVC
# models que probem per seleccionar:
models = [
    RandomForestClassifier(n_estimators=100, random_state=0), # eliminem: max_depth=5
    LinearSVC(),
    MultinomialNB(),
    GaussianNB(),
    LogisticRegression(random_state=0),
    SVC(C=10, kernel="linear"),
    #NuSVC(nu=2, kernel='linear')
]

# Cross-validation k-fold amb k=5:
k = 5

# PROBAR: sklearn.pipeline.Pipeline(steps, *, memory=None, verbose=False)

entries = []
for model in models:
    model_name = model.__class__.__name__
    start_time = time()
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=k)
    elapsed_time = time() - start_time # Temps d'execució del model amb el CV on k=5.
    print('Temps de execució del {} = {}'.format(model, elapsed_time))
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
cv_df

Temps de execució del RandomForestClassifier(random_state=0) = 16.81508779525757
Temps de execució del LinearSVC() = 1.820134162902832
Temps de execució del MultinomialNB() = 0.5964045524597168
Temps de execució del GaussianNB() = 4.7373340129852295
Temps de execució del LogisticRegression(random_state=0) = 14.116254091262817
Temps de execució del SVC(C=10, kernel='linear') = 243.06422066688538


,model_name,fold_idx,accuracy
0,RandomForestClassifier,0,0.859206
1,RandomForestClassifier,1,0.823105
2,RandomForestClassifier,2,0.841155
3,RandomForestClassifier,3,0.801444
4,RandomForestClassifier,4,0.847826
5,LinearSVC,0,0.931408
6,LinearSVC,1,0.906137
7,LinearSVC,2,0.913357
8,LinearSVC,3,0.888087
9,LinearSVC,4,0.916667


In [38]:
# Mesurem el model amb el millor accuracy mitjà:
mac = cv_df.groupby('model_name').accuracy.mean()
mac.columns = ['Mean Accuracy']
mac

model_name
GaussianNB                0.711722
LinearSVC                 0.911131
LogisticRegression        0.885839
MultinomialNB             0.856221
RandomForestClassifier    0.834547
SVC                       0.898130
Name: accuracy, dtype: float64

Ara probarem amb el Pipeline quin accuracy ens dona el LinearSVC per comprobar si ens dona millors resultats que utilitzant el cross-validation:

In [52]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.datasets import make_classification

X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state = 0) # 70% train, 30% test: test_size=0.3,
# Podem probar altres mesures (80%,20% o 50%,50%)

pipe = Pipeline([('scaler', StandardScaler()), ('Linearsvc', LinearSVC())])
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)


C:\Users\oscar\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.8872832369942196

Podem observar que aplicant el pipeline ens dona un resultat una mica mes baix que amb el cross-validation i a part triga molt de temps la execució d'aquest.